In [1]:
import pandas as pd
import xarray as xr
import numpy as np

from bluemath_tk.datamining.pca import PCA
from bluemath_tk.interpolation.rbf import RBF

In [2]:
#### Load project Assests

mda_df = pd.read_csv("assets/mda_df.csv", index_col=0)
case_library = xr.open_dataset(
    "/lustre/geocean/WORK/users/Pablo/XBeachCases/case_library.nc"
)

In [3]:
vars_rec = ["zs_mean", "zs_var"]
vars_mda = ["Hs", "Steepness", "Dir", "SWL"]

#### Principal Component Analysis And Radial basis Function interpolation

In [4]:
case_library = case_library.assign_coords(
    {
        "y": ("y", np.arange(len(case_library.y))),
        "x": ("x", np.arange(len(case_library.x))),
    }
).set_coords(["y", "x"])  # Make them coordinates, not dimensions

n_cells_remove_water = 50
n_cells_remove_sides_1 = 40
n_cells_remove_sides_2 = 30

case_library = case_library.isel(
    x=np.arange(n_cells_remove_water, len(case_library.x), 1)
)  # Remove water contour
case_library = case_library.isel(
    y=np.arange(n_cells_remove_sides_1, len(case_library.y) - n_cells_remove_sides_2, 1)
)  # Remove lateral contour

globalx_values = case_library.globalx.values
globaly_values = case_library.globaly.values

In [5]:
for var in vars_rec:
    
    pca = PCA(
        n_components=0.99,
        is_incremental=False,
        debug=False,
    )

    pca.fit(
        data=case_library,
        vars_to_stack=[var],
        coords_to_stack=["y", "x"],
        pca_dim_for_rows="case_num",
    )

    pcs = pca.transform(data=case_library)
    pcs.to_netcdf(f"assets/pca_rfb_objects/{var}_pcs.nc")
    pca.save_model(f"assets/pca_rfb_objects/{var}_pca_ob.pkl")

    rbf = RBF()

    target = pd.DataFrame(pcs["PCs"].values)

    rbf.fit(
        subset_data=mda_df[["Hs", "Steepness", "Dir", "SWL"]],
        subset_directional_variables=[],
        target_data=target,
        normalize_target_data=True,
        iteratively_update_sigma=True,
    )

    rbf.save_model(f"assets/pca_rfb_objects/{var}_rbf_ob.pkl")


        -------------------------------------------------------------------
        | Initializing PCA reduction model with the following parameters:
        |    - n_components: 0.99
        |    - is_incremental: False
        | For more information, please refer to the documentation.
        -------------------------------------------------------------------
        

        ---------------------------------------------------------------------------------
        | Initializing RBF interpolation model with the following parameters:
        |    - sigma_min: 0.001
        |    - sigma_max: 0.1
        |    - sigma_diff: 0.0001
        |    - sigma_opt: None
        |    - kernel: gaussian
        |    - smooth: 1e-05
        | For more information, please refer to the documentation.
        | Recommended lecture: https://link.springer.com/article/10.1023/A:1018975909870
        ---------------------------------------------------------------------------------
        

        -----